In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("D7.ipynb")

# Discussion 7: Support Vector Machines

Support vector machines (SVMs) are a particularly powerful and flexible class of supervised algorithms for both classification and regression.
In this notebook, we will develop the intuition behind support vector machines and their use in classification problems.

SVMs are powerful classification method for several reasons:

- They rely on relatively few support vectors, making them compact and memory-efficient.
- They offer fast prediction once the model is trained.
- They work well with high-dimensional data, even when the number of dimensions exceeds the number of samples.
- Their versatility is enhanced by their integration with kernel methods, enabling them to adapt to various data types.

However, SVMs also have some drawbacks:

- The computational cost scales with the number of samples, which can be a limitation for large datasets.
- The choice of the regularization parameter C significantly impacts the results and must be carefully selected via cross-validation, which can be expensive for larger datasets.
- The results lack a direct probabilistic interpretation.


## Instructions:

Just because you pass the public tests **does not mean you will pass hidden tests**. Public tests are there to guide your implementation, but the correct answer is hidden.

Do NOT change any of the given code.

Read the markdown cells carefully, as they'll provide hints towards writing your solutions. For each question, you'll be expected to import the required packages and implement based upon the description. All necessary information will be provided, so again, read carefully!




## Motivation of Support Vector Machines

Consider the simple case of a classification task, in which the two classes of points are well separated:

In [ ]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# use seaborn plotting defaults
import seaborn as sns; sns.set()
from sklearn.datasets import make_blobs


X, y = make_blobs(n_samples=100, centers=2,
                  random_state=18, cluster_std=1)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='Spectral');

A linear discriminative classifier seeks to create a classification model by drawing a straight line that separates two sets of data. When dealing with two-dimensional data, such as the example shown here, this task can be done manually. However, a challenge arises because there are multiple possible dividing lines that can accurately discriminate between the two classes. We can illustrate this by drawing different lines.

In [ ]:
xfit = np.linspace(0, 10)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='Spectral')
plt.plot([3], [-4], 'o', color='black', markeredgewidth=4, markersize=12)

for m, b in [(1, -8), (0.4, -6), (-0.2, -2)]:
    plt.plot(xfit, m * xfit + b, '-k')

These three separators are remarkably distinct, yet they all effectively separate the samples. Depending on the decision boundary chosen, a new data point (indicated by the black point in the plot) will be classified differently. It is clear that relying solely on the intuition of "drawing a line between classes" is insufficient, and we must delve deeper to construct a classifier that will generalize best to unseen data.

## Support Vector Machines: Maximizing the *Margin*

Support vector machines offer one way to improve on this.
The intuition is this: rather than simply drawing a zero-width line between the classes, we can draw around each line a *margin* of some width, up to the nearest point.
Here is an example of how this might look:

In [ ]:
xfit = np.linspace(0,10)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='Spectral')
plt.plot([3], [-4], 'o', color='black', markeredgewidth=4, markersize=12)
d = 0.3
for m, b in [(1, -8), (0.4, -6), (-0.2, -2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='#AAAAAA', alpha=0.8)

plt.xlim(0, 10);

### Question 1: Fitting a Support Vector Machine

Support vector machines serve as a maximum margin estimator; the optimal model in support vector machines is determined by selecting the line that maximizes the margin.

To get started, you will implement a linear Support Vector Classifier by importing https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html and fitting it on the entire `X` and `y` data generated above.


In this section, try a very large regularization value of 1000.

We include a helper function to plot the SVM decision boundaries.

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
# Import Necessary Packages
...
# Define and Fit your Model
model = ...
...

# Plot the Data and the Decision Boundary
...
...

This is the dividing line that maximizes the margin between the two sets of points.
Notice that a few of the training points just touch the margin: they are indicated by the black circles in this figure.
These points are the pivotal elements of this fit, and are known as the *support vectors*, and give the algorithm its name.
In Scikit-Learn, the identity of these points are stored in the ``support_vectors_`` attribute of the classifier:

In [ ]:
grader.check("Fit SVM")

## Question 2: Support Vectors

The success of this SVM classifier is attributed to the fact that, for the fitting process, only the position of the support vectors is significant. Points that are correctly classified and located further away from the margin do not alter the fit. This is due to the fact that these points do not contribute to the loss function utilized for fitting the model, and thus their position and quantity are irrelevant as long as they do not cross the margin.

This phenomenon is evident when examining the models learned from the first 60 points and the first 120 points of the dataset.

**Hint**: Your decision boundaries should look the same!

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.70)
    # Select N points
    X = ...
    y = ...
    
    # Fit an SVC again with a large value of C
    model = ...
    ...
    
    # Plot the data using ax.scatter
    if ax:
        ...
    
        # Call the helper function defined about for plotting the decision boundary
        ...
    
    return model
    

    
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, N in zip(ax, [60, 100]):
    model = plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

The model and support vectors for 60 training points are visible in the left panel; the right panel shows that we have increased the number of training points by two, but the model remains the same, and the three support vectors from the left panel are still the support vectors from the right panel. This ability to remain unaffected by distant points' behavior is one of the SVM model's advantages.

In [ ]:
grader.check("Support Vectors")

### Question 3: Kernel SVM

SVM exhibits its potential when it is utilized beyond the linear case, with kernels. To illustrate the necessity of kernels, let's examine some data that cannot be separated linearly.

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1, random_state=42)

model = SVC(kernel='linear')
model.fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='Spectral')
plot_svc_decision_function(model, plot_support=False);

It's apparent that linear discrimination can never segregate this data. However, we can elevate the data to a higher dimension, making a linear separator feasible. For instance, a straightforward projection we can employ is calculating a radial basis function, centered on the central cluster:

In [ ]:
r = np.exp(-(X ** 2).sum(1))

In [ ]:
from mpl_toolkits import mplot3d

def plot_3D(r, elev=30, azim=30, X=X, y=y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

plot_3D(r, X=X, y=y)

However, selecting and adjusting our projection requires careful consideration. If we do not correctly center our radial basis function, we cannot achieve such neat and linearly separable results. In general, this selection process poses a problem, and we need to find the optimal basis functions automatically.

One strategy to this end is to compute a basis function centered at *every* point in the dataset, and let the SVM algorithm sift through the results.
This type of basis function transformation is known as a *kernel transformation*, as it is based on a similarity relationship (or kernel) between each pair of points.

However, projecting $N$ points into $N$ dimensions— might become very computationally intensive as $N$ grows large.
However, because of a neat little procedure known as the [*kernel trick*](https://en.wikipedia.org/wiki/Kernel_trick), a fit on kernel-transformed data can be done implicitly—that is, without ever building the full $N$-dimensional representation of the kernel projection!
This kernel trick is built into the SVM, and is one of the reasons the method is so powerful.

Implement a kernelized SVM by swapping the linear kernel for an RBF (radial basis function) kernel.

_Type your answer here, replacing this text._

In [ ]:
# Fit an SVM with an RBF kernel and C=1000
model = ...
...

In [ ]:
...
...

By employing this kernelized support vector machine, we can develop an appropriate nonlinear decision boundary. This kernel transformation approach is frequently used in machine learning to convert rapid linear methods into efficient nonlinear methods, primarily for models that can employ the kernel trick.

In [ ]:
grader.check("Kernel SVM")

### Question 4: Margins

In perfectly clean, simulated datasets a perfect decision boundary exists. But what about noisier data that you might encounter in the real world?

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='Spectral');

In the SVM implementation, the hardness of the margin is controlled by a regularization hyperparameter, most often known as $C$.

Large values of $C$ construct SVM with a hard margin. Points can not lie within the margin.
Small values of $C$, construct SVM with a soft margin. Points can lie wiwthin the margin.

Split your data, `0.8` in train, `0.2` in test, `random_state=42`. Train two separate models using the given values of `C` $10.0$ and $0.1$. Append your test set performance to the `test_accuracies` list.
Generate two plots demonstrating this characteristic of SVMs.

You may use the `train_test_split` and `accuracy_score` functions from sklearn.

In [ ]:
# Import Packages
...
...

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)


...
test_acc = ...

for axi, C in zip(ax, [10.0, 0.1]):
    # Define your model
    model = ...
    
    # Calculate and append Accuracy
    y_pred = ...
    accuracy = ...
    ...
    
    # Plotting
    ...
    ...
    axi.set_title('C = {0:.1f}'.format(C), size=14)

In [ ]:
grader.check("Soft Margins")

The optimal value of the $C$ parameter will depend on your dataset, and should be tuned using cross-validation or a similar procedure (refer back to [Hyperparameters and Model Validation](05.03-Hyperparameters-and-Model-Validation.ipynb)).